In [16]:
!pip install surprise
import pandas as pd
import numpy as np

import surprise
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings; warnings.simplefilter('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [42]:
rootdir = '/content/drive/My Drive/Colab Notebooks/Movie  Recommendation System/Movie Dataset/'

In [43]:
ratings = pd.read_csv(rootdir+'ratings.csv')

In [44]:
ratings.shape

(100836, 4)

In [45]:
movies = pd.read_csv(rootdir+'movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


**Data Preprocessing**

In [46]:
for dataset in [ratings,movies]:
  dataset.drop_duplicates(inplace=True)

In [47]:
ratings[ratings['userId'].isnull()]

,movieId,title,userId,rating


In [24]:
ratings[ratings['rating'].isnull()]

,movieId,title,userId,rating


**CF based recommendation system**

Our content based engine suffers from some severe limitations.

* It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.
* Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who (s)he is.
* Therefore, in this section, we will use Collaborative Filtering to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.


**User based**

In [48]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [58]:
userRatings = userRatings.replace(0,np.nan)

In [59]:
userRatings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [82]:
def normalize(row):
    new_row = (row - np.nanmean(row))
    return new_row

* since each user has its own standard we can't assume A's rating 7 is same as another users' 7. So we use Correlation similarity. We use Shifted Cosine similarity also known as Pearson correlation

In [83]:
shiftedratings = userRatings.apply(normalize, axis=1)
shiftedratings = shiftedratings.replace(np.nan,0)
shiftedratings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,-0.383886,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.616114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.616114,0.0,0.0,-1.383886,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-0.383886
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,1.019231,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,-0.980769,0.000000,0.000000,0.00000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,-0.979167,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,-0.979167,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.407821,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.00

In [78]:
cosine_sim = cosine_similarity(shiftedratings,shiftedratings)
cosine_sim

array([[ 1.00000000e+00,  5.67030302e-04,  3.57561442e-02, ...,
         8.81124927e-02, -3.43779708e-02,  9.50087619e-03],
       [ 5.67030302e-04,  1.00000000e+00,  0.00000000e+00, ...,
        -9.19819487e-03, -8.76800175e-02,  3.89118580e-02],
       [ 3.57561442e-02,  0.00000000e+00,  1.00000000e+00, ...,
        -1.75398073e-02,  0.00000000e+00,  2.17727443e-02],
       ...,
       [ 8.81124927e-02, -9.19819487e-03, -1.75398073e-02, ...,
         1.00000000e+00,  5.36559226e-02,  4.91659617e-02],
       [-3.43779708e-02, -8.76800175e-02,  0.00000000e+00, ...,
         5.36559226e-02,  1.00000000e+00, -2.22672505e-02],
       [ 9.50087619e-03,  3.89118580e-02,  2.17727443e-02, ...,
         4.91659617e-02, -2.22672505e-02,  1.00000000e+00]])

In [79]:
resultDf = pd.DataFrame(cosine_sim,index=shiftedratings.index,columns=shiftedratings.index)

In [80]:
resultDf

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000567,0.035756,0.054725,0.022255,-0.053379,-0.009250,0.049455,0.025576,-0.009141,-0.061919,0.014717,0.065546,0.023631,0.037707,0.011813,0.037425,0.048299,0.104639,0.072433,0.015754,-0.005219,-0.029329,0.006772,-0.002078,-0.016131,0.055866,0.000960,-0.007781,0.003171,0.006602,0.044456,0.013691,0.010260,0.022960,0.030076,-0.052408,0.009268,-0.079442,-0.045394,...,-0.008322,0.052842,0.014307,0.042304,0.020211,-0.000639,0.100014,0.000000,-0.116520,0.023104,0.007601,0.009816,-0.006730,-0.011990,0.010732,0.013863,-0.048843,-0.020318,-0.063654,0.086838,0.044852,-0.064779,0.087121,0.016216,0.037431,0.041397,0.111593,0.007337,0.064518,0.081834,0.018758,-0.019902,-0.021318,-0.039942,-0.041026,0.015838,0.059647,0.088112,-0.034378,0.009501
2,0.000567,1.000000,0.000000,-0.020703,0.026409,-0.027165,-0.011773,-0.058264,0.000000,0.004909,-0.003734,0.000000,0.001410,0.015880,-0.035536,-0.018798,-0.001898,-0.073402,-0.000105,0.000272,-0.004226,-0.034139,-0.000595,-0.046996,0.015986,0.000000,0.000000,-0.007065,-0.025531,-0.043542,0.000000,-0.040169,0.010702,-0.013307,0.000000,-0.045829,0.009459,-0.052009,0.000000,-0.032161,...,0.000000,-0.008892,0.025063,-0.064598,0.000000,0.000000,-0.037497,0.000000,0.000000,-0.014730,-0.085375,-0.034829,-0.000738,0.000000,0.012024,-0.015874,0.000000,-0.060046,-0.043218,0.001239,0.001126,-0.010735,-0.032976,0.000121,0.000000,0.038165,0.006659,0.001268,-0.004087,-0.002098,-0.064763,-0.038284,-0.002307,0.000000,0.000000,0.008570,-0.025371,-0.009198,-0.087680,0.038912
3,0.035756,0.000000,1.000000,-0.009702,-0.025852,0.007724,0.000000,-0.026616,0.000000,0.000000,0.000000,0.000000,0.000000,0.007254,-0.068686,0.009997,-0.003844,-0.015065,-0.014376,-0.016671,-0.060234,0.008708,-0.028286,-0.007730,-0.012787,0.000000,-0.004066,0.010381,-0.014115,0.000000,-0.001255,-0.096341,-0.026700,0.023777,0.000000,0.000000,0.000000,-0.023759,0.024279,-0.002600,...,-0.046610,-0.004280,0.000000,0.000000,0.000000,-0.114745,-0.015859,0.000000,0.001940,0.018116,-0.030998,0.000000,0.000000,0.000000,0.000000,0.026874,-0.011230,-0.011369,-0.019743,-0.040129,0.000000,0.000000,0.008651,-0.008206,0.000000,0.042052,-0.011725,0.000000,0.023485,-0.001062,-0.003876,-0.012896,0.037607,0.000000,-0.006631,-0.068387,-0.023505,-0.017540,0.000000,0.021773
4,0.054725,-0.020703,-0.009702,1.000000,-0.033639,0.016360,0.066016,0.001969,-0.007315,0.063340,0.067985,-0.016834,-0.072042,-0.062982,0.035582,-0.028462,-0.041741,-0.006701,0.006026,0.040447,-0.001164,-0.008101,-0.017091,-0.010940,-0.010326,-0.020886,-0.002374,0.017392,0.036471,0.000182,-0.009489,0.058247,0.013852,0.051930,0.004566,-0.004342,-0.017524,0.023511,0.024636,-0.013678,...,-0.000185,-0.017048,-0.001261,0.034540,-0.039494,-0.027277,0.067531,0.000000,-0.020832,-0.095417,0.053056,-0.028718,-0.039405,-0.009709,0.004051,-0.019723,-0.046194,0.007690,-0.010178,0.094020,-0.017187,-0.019032,0.061966,-0.016432,0.008271,-0.006291,0.046073,-0.013355,0.016394,0.031047,-0.042859,0.067496,0.019373,-0.015518,0.047363,0.023584,0.017039,-0.044062,-0.024162,0.000050
5,0.022255,0.026409,-0.025852,-0.033639,1.000000,0.014214,0.010516,-0.012284,0.000000,-0.036717,-0.013343,0.066440,0.016915,-0.037283,-0.015385,-0.033486,0.052430,0.023270,-0.009796,0.045758,-0.005934,-0.022805,-0.012358,0.003355,0.021805,0.104230,0.007858,0.026829,-0.025385,0.034557,0.032236,0.066080,0.061390,0.039661,0.301256,-0.019814,0.135083,0.160828,-0.024993,0.017411,...,-0.020448,0.036193,0.032989,0.038439,0.069674,0.000000,0.110031,0.000000,-0.003308,0.005693,0.023493,0.000000,-0.032672,0.117091,0.031924,-0.048724,0.099128,0.129691,0.102781,0.053524,0.000000,-0.0

In [81]:
def get_similar_users(user):
    if user not in resultDf.index:
        return None
    else:
        similarList = zip(resultDf[user],list(resultDf.columns.values))
        similarList = sorted(similarList, key=lambda x: x[0], reverse=True)
        return similarList[1:10]
get_similar_users(609)

[(0.34170839259587626, 565),
 (0.33256807812105643, 242),
 (0.32409441086466756, 455),
 (0.2979243814969141, 102),
 (0.29216699339285174, 530),
 (0.2863365588437133, 54),
 (0.283736743536024, 444),
 (0.2820350719859673, 512),
 (0.24842702325531668, 589)]